In [3]:
%pip install transformers torch pandas
%pip install sentencepiecpip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl (797.1 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

# Load data from CSV
file_path = 'cleaned_infopankki-fa.csv'
data = pd.read_csv(file_path)

# Display the first few rows to verify the structure
print(data.head())

# Assuming the columns are 'English' and 'Persian'
english_texts = data['English'].tolist()
persian_texts = data['Persian'].tolist()

                                             English  \
0  all texts that have been published on the info...   
1                                    you are free to   
2  share copy and redistribute the material in an...   
3  adapt remix transform and build upon the mater...   
4                          under the following terms   

                                             Persian      Source  
0  تمام مطالب و متونی که به هر زبانی در صفحات این...  infopankki  
1                                    شما اجازه دارید  infopankki  
2  به اشتراک بگذارید از مطالب نسخه برداری کنید و ...  infopankki  
3  تغییر دهید مطالب را ترکیب کنید و تغییر دهید و ...  infopankki  
4                                        با شروط زیر  infopankki  


In [7]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the mBART tokenizer and model
model_name = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set the target language (Persian)
tokenizer.src_lang = "en_XX"
target_lang = "fa_IR"

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [ ]:
import torch

def translate_text(text, model, tokenizer, target_lang):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Generate translation using the model
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])

    # Decode the generated tokens into a string
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

# Translating English sentences to Persian
translations = []
for sentence in english_texts:
    translated_sentence = translate_text(sentence, model, tokenizer, target_lang)
    translations.append(translated_sentence)

# Adding translations to the dataframe
data['mBART_Persian'] = translations

# Save the dataframe to a new CSV
data.to_csv('translated_infopankki.csv', index=False)
